# ニューラルネットワークの数学 2 （逆伝播）

本章では、第 1 章で学んだ順伝播の計算から平均二乗誤差などの損失から、どのようにニューラルネットワークのパラメータの更新を行うのかについて学びます。次章の実装に移る前にニューラルネットワークの学習の一連の流れを理解する事をゴールとします。  

## 本章の構成

- 勾配降下法
- ミニバッチ学習
- パラメータの更新量の算出

## 勾配降下法

もう一度ニューラルネットワークの計算の流れを確認しましょう。  

![ニューラルネットワークの計算](http://drive.google.com/uc?export=view&id=1uCbsMBbU0HYQskN7HGhHRnr_QvZ6gPms)

1. パラメータの初期値をランダムに決定
2. 順伝播の計算 : 予測値 $y$ の算出
3. 損失の計算 : 損失 $\mathcal{L}$ の算出
4. 勾配の計算 : 勾配の算出
5. パラメータの更新 : 勾配を用いて重みを更新
6. 2 ~ 5 を繰り返し、最適なパラメータに調整を行う

前章では 1 ~ 3 までの順伝播そして、損失の計算方法について確認しました。4 ステップ目の勾配の計算方法は誤差逆伝播法とも呼ばれます。その計算方法は少し複雑であり、TensorFlow などのフレームワークを用いてニューラルネットワークを実装する際にはフレームワーク側でラップされているため特に意識する必要がありません。そのため本講座では誤差逆伝播法の計算方法について詳しくは取り扱いません。  

本章では 5 ステップ目のパラメータの更新方法についてまず解説します。  


### ニューラルネットワークのパラメータの最適化

重回帰分析のパラメータの最適化では、目的関数をパラメータで微分して勾配ベクトルを求め、そしてその勾配ベクトルがゼロベクトルとなるように、各パラメータの値を求めることによって行っていました。  

ニューラルネットワークのパラメータの最適化に対し、この方法を採用した場合に 1 つ問題があります。それは、今回の目的関数を計算する中で ReLU 関数を採用する場合に  $\max(0, x)$ が含まれていることです。この関数は実際に値を入れていなければ関数の値を決定することができません。つまり、重回帰分析のように全体を見通して目的関数の微分を行うことができません。微分を行うことができなければ、パラメータの最適化を行うことが出来ません。  

そのため、ニューラルネットワークでは、パラメータの初期値をランダムにとり、順伝播を通して実際に計算を行い、損失を計算することで、微分を可能にしています。そして、求まった傾きを用いて、パラメータを更新してくという流れとなっています。  

### 勾配降下法のアルゴリズム

ニューラルネットワークのパラメータの更新に用いられる最適化のアルゴリズムの 1 つに **勾配降下法** があります。本節では、勾配降下法のアルゴリズムの理解を通して、ニューラルネットワークのパラメータの最適化方法について学びます。  

まず、以下の図を見てください。  
図中の線は、パラメータ $w$ を変化させた際の目的関数 $\mathcal{L}$ の値を表しています。  
この例では簡単のため二次関数の形になっていますが、ニューラルネットワークの目的関数は実際には多次元で、かつもっと複雑な形をしていることがほとんどです。
しかし、ここでは説明のためこのようにシンプルな形を考えます。
さて、この目的関数が最小値を与えるような $w$ は、どのようにして発見できるでしょうか。

![勾配降下法 1](http://drive.google.com/uc?export=view&id=1xpBugThwUugFcgLatBAFo2-0deyEUTpW)

さきほど説明したように、ニューラルネットワークのパラメータはまず乱数で初期化されます。  ここでは、例として $w=8$ という初期化が行われた場合を想定します。   
そうすると、$w=8$ における $\mathcal{L}$ の勾配 $\frac{\partial \mathcal{L}}{\partial w}$ が求まります。ニューラルネットワークの目的関数は、全てのパラメータについて微分可能であるように設計されることに注意してください。  

ここでは仮に $w=8$ における $\frac{\partial \mathcal{L}}{\partial w}$ が $5$ であったとします。  
以下の図のように、この $5$ という値は $w=8$ における $\mathcal{L}(w)$ という関数の接線の傾き、勾配（gradient）を表しています。  

![勾配降下法 2](http://drive.google.com/uc?export=view&id=1-ekRMZGQbgTDMI39ASfOVAaoaAmCbUia)


傾きとは、$w$ を増加させた際に $\mathcal{L}$ が増加する方向を意味しているので、今は $\mathcal{L}$ の値を小さくしたいわけですから、この傾きの逆方向へ $w$ を変化させる、すなわち $w$ **から** $\frac{\partial \mathcal{L}}{\partial w}$ **を引けばよい** ことになります。  

式であらわすと下記になります。

$$
w \leftarrow w - \dfrac{\partial \mathcal L}{\partial w}
$$

左側の $w$ はパラメータ更新後の $w$ になります。そして、矢印の右側はパラメータの更新方法を表しています。前述のとおり、最初のパラメータ $w$ から勾配を引いています。今回の数値例に当てはめると、  

$$
3 \leftarrow 8 - 5
$$

となります。  
これがニューラルネットワークのパラメータを目的関数の勾配を用いて更新していく際の基本的な考え方です。  
下記の図ではパラメータ更新の一連の流れを繰り返しています。図を確認し、より具体的なイメージを使いましょう。  

![勾配降下法 3](http://drive.google.com/uc?export=view&id=1Xni4-MEr6QhVqs0J9yvA7XYxhqd7GvbK)

徐々に $\mathcal{L}$ が最小値をとるときの $w$ の値に近づいていることが見て取れます。

### 学習率

ニューラルネットワークのパラメータの最適化について理解するためには**学習率 (Learning rate)** について理解しておく必要があります。パラメータの更新量を決定する際に、パラメータから勾配を引くという計算を行いましたが、実際には勾配に対して学習率と呼ばれる値を乗じるのが一般的です。下記は学習率を $lr$ とした場合の式になります。  

$$
w \leftarrow w - lr \dfrac{\partial \mathcal L}{\partial w}
$$

実際の学習時には勾配をそのままパラメータを更新量とすると値が大きすぎる場合が一般的です。そのため、0.001 などの学習率を乗じることでパラメータの更新量を調整します。学習率は一種のハイパーパラメータと呼ぶことができ、モデルの学習結果を確認しながら調整を行います。学習率が大きすぎる場合、小さすぎる場合のデメリットを確認しましょう。  

![学習率](http://drive.google.com/uc?export=view&id=1iJIS4K1rBqf0yfPDdG1yo_RHFoUgpsLq)

- 学習率が小さすぎる場合 : パラメータの更新量が小さくなりすぎ、学習の収束までに時間を要する
- 学習率が大きすぎる場合 : パラメータの調整が収束せず学習がうまく進まない

実装を行う際にはこの学習率の調整も必要な場合があることを覚えておきましょう。  

## ミニバッチ学習

通常ニューラルネットワークを勾配降下法で最適化する場合は、データを一つ一つ用いてパラメータを更新するのではなく、いくつかのデータをまとめて入力し、それぞれの勾配を計算したあと、その勾配の平均値を用いてパラメータの更新を行う方法がよく行われます。 これを**ミニバッチ学習**と呼びます。  

下記の図を見てミニバッチ学習を理解しましょう。  

![ミニバッチ学習](http://drive.google.com/uc?export=view&id=1qvhdR_f69GCAdkE3pfcXiLbmV0N2rP34)

ミニバッチ学習時のパラメータの更新の流れを確認します。先程の勾配降下法との違いについて着目して読み進めて下さい。  

1.  指定した数のサンプルを抽出（図内は 120 サンプルを 3 つずつにまとめる）
2. (1) で抽出したサンプルごとの損失を計算（図内では $\bf　x_1$~$\bf　x_3$）
3. 損失の平均を計算
4. 損失の平均から勾配を計算
5. パラメータの更新

※ 1. のサンプルの抽出は基本的にランダムに抽出されます。

上記のミニバッチ学習と異なり、データセット全体を一度に用いてパラメータの更新を 1 回を行う（抽出を行わない）方法を**バッチ学習**と呼ぶことも覚えておいて下さい。  

勾配降下法では、データを用いてパラメータを 1 度更新するまでのことを **1 イテレーション (iteration)** と呼び、バッチ学習では学習用データセットのデータ全体を 1 度の更新に用いるため、**1 エポック = 1 イテレーション** となります。 一方、例えばデータセットを 10 個のグループに分割してグループごとに更新を行うミニバッチ学習の場合は、**1 エポック = 10 イテレーション** となります。

図内の例を上記の言葉に置き換えると次になります。  

- サンプル数 : 120
- バッチサイズ : 3
- イテレーション : 40 (120/3)
- （エポック : 1）

バッチサイズとエポック数は実装する際にも調整が必要なものになります。それぞれがどのような意味を持つのかしっかり理解しておきましょう。  
このようなミニバッチ学習を用いた勾配降下法は特に、**確率的勾配降下法（stocastic gradient descent; SGD）**と呼ばれます。 現在多くのニューラルネットワークの最適化手法はこの SGD をベースとした手法となっています。

## 誤差逆伝播法

これまで、勾配を用いてニューラルネットワークがどのようにパラメータを更新するのかを確認しました。本節では、どのように損失から勾配を算出するかの基礎的な考え方について理解します。  



### 合成関数の微分

ニューラルネットワークの勾配の算出を行うためには合成関数の微分の方法について理解する必要があります。順伝播で確認したニューラルネットワークを例題に計算方法を解説します。

![合成関数の微分](http://drive.google.com/uc?export=view&id=1xUHUJh2sY9RkL0YIS2yDUie4M7SmqKoB)

目的関数に対して、更新したいパラメータで偏微分を行うことで更新量を算出できるのですが、複数の層にまたがって複雑な計算が行われているため、そのままでは求めることが困難です。そこで、多層のニューラルネットワークでは、何層にも重ねた層の一つ一つの 1 つの関数をみなすことで、多層のニューラルネットワークは複数の関数（層）を合成した**合成関数**と見ることができます。  

合成関数の中身を解説していきます。関数 $y = f(x)$ と $z = g(y)$ の**合成**とは、 $f$ を適用したあとに $g$ を適用する関数、$z = g(f(x))$ を指します。多層のニューラルネットワークでは、この合成関数を微分することを考えるため、**連鎖律 (chain rule)** と呼ばれる合成関数の微分公式を使用します。連鎖律は、合成関数の微分を簡単に計算するための公式だけではなく、ニューラルネットワークの学習方法である**誤差逆伝播法**を理解する上で本質的な役割を果たします。  

今回は簡単な例として、

$$
\begin{align}
f(x) = \bigl\{(2x+4)^2 \bigr\} ^{'}
\end{align}
$$

を計算することにします。この式を計算しようとすると最初に思い浮かぶのは $(4x^2 + 16x + 16)^{'}$ とカギ括弧の中身を展開してからそれぞれを微分することです。その方法でも計算できるのですが、さらに複雑な計算の場合 3 乗や 4 乗という風に式展開することが大変になります。ここで合成関数の微分が役に立ちます。合成関数の微分では、内側の部分である $2x+4$ と外側の部分である $(\cdot)^2$ をそれぞれ微分を行い、その結果を掛け合わせることで求めます。  

それでは、具体的に $f(x)$ という関数の微分を考えてみます。内側の関数を $u = 2x+4$ とおくと、

$$
\begin{align}
f(x) &= u^2 \\\\
u &= 2x+4
\end{align}
$$

とできます。ここで合成関数の微分から求めたい式をまとめると、

$$
\begin{align}
\frac{\partial}{\partial x}f(g(x))  = \frac{\partial f(x)}{\partial u}\frac{\partial u}{\partial x}
\end{align}
$$

上の式で $u = g(x)$ です。それでは順番に計算していきましょう。  

$$
\begin{align}
\frac{\partial f(x)}{\partial u} &= \frac{\partial }{\partial u} (u^{2}) \\\\
&= 2u \\\\
&= 2(2x+4)
\end{align}
$$

$$
\begin{align}
\frac{\partial u}{\partial x} &= \frac{\partial }{\partial u}2x+4  \\\\
&=2
\end{align}
$$

上記で計算されたそれぞれの値をかけ合わせる事により、結果を獲得することができます。

$$
\begin{align}
\frac{\partial }{\partial x}f(g(x)) &= \frac{df(x)}{\partial u}\frac{\partial u}{\partial x} \\\\
&= 2(2x+4) \cdot 2 \\\\
&= 8x+16
\end{align}
$$

計算できました。$(4x^2 + 16x + 16)^{'}$ と展開してから微分する方法と結果が一致しています。このような合成関数の微分公式を使用して、ニューラルネットワークではパラメータの更新量を求めます。

### パラメータの更新量の算出

それでは実際に合成関数の微分公式を使用して、先程の例で実際にパラメータの更新量を計算していきましょう。

![20_3](http://drive.google.com/uc?export=view&id=15hMMfWDWu-YnfsaAwZfRe8CvXdtRUvCL)

上の図の $w_7$ における勾配を計算します。同じ層にあるパラメータは同じ手順で求めることができます。  

目的関数に対する $w_7$ の勾配 $\frac{\partial L}{\partial w_7}$ を求めます。（$\bf u_1$ 、$\bf z_1$ 、$y$ 、$\mathcal L$ は事前に計算が完了しているものとします。非線形変換には ReLU 関数を使用します。）目的関数 $L$ の式から微分したい $w_7$ の式までの流れを確認しておきましょう。まず、回帰問題の目的関数は、

$$
\begin{align}
{\mathcal L} = \frac{1}{N} \sum_{n=1}^{N} (t_n - y_n)^{2}
\end{align}
$$

で表される平均ニ乗誤差でした。また、予測値 $y_n$ は、

$$
\begin{align}
y_n = w_7z_{11} + w_8 z_{12} + b
\end{align}
$$

となります。  

今回はわかりやすいようにサンプル数 1 として考え、バイアスに関しては考慮しないとします。これより目的関数 $\mathcal L$ は、

$$
\begin{align}
{\mathcal L} &= (t - y)^{2} \\\\
&= \bigl\{t - (w_7z_{11} + w_8 z_{12})\bigr\}^{2}
\end{align}
$$

となります。それでは求めたい式を整理しておくと、

$$
\begin{align}
\frac{\partial \mathcal L}{\partial w_7} = \frac{\partial \mathcal  L}{\partial y}\frac{\partial y}{\partial w_7}
\end{align}
$$

とそれぞれの式に対応する微分へと切り分けることができるので、前節で学んだ合成関数の微分を用いながら計算していくと、

$$
\begin{align}
\frac{\partial L}{\partial y} &= 2(t - w_7z_{11} - w_8 z_{12}) \\\\
\frac{\partial y}{\partial w_7} &= - z_{11}
\end{align}
$$

より、求めたい勾配の値は、

$$
\begin{align}
\frac{\partial \mathcal  L}{\partial w_7} &= \frac{\partial \mathcal  L}{\partial y}\frac{\partial y}{\partial w_7} \\\\
&= 2(t - w_7z_{11} - w_8 z_{12}) × (-z_{11}) \\\\
&= -2 × 11 (10 - 3 × 11 - 2 × 0) \\\\
&= 506
\end{align}
$$

となります。勾配降下法から、学習係数 $lr = 0.01$ の場合、更新後の $w_7$ の値は、

$$
\begin{align}
w_7 &= w_7 - lr\frac{\partial \mathcal  L}{\partial w_7} \\\\
&= 3 - 5.06 \\
&= -2.06
\end{align}
$$

となります。  

今回は出力層にもっとも近いパラメータの更新方法をお伝えしましたが、層がさらに深くなるとこの計算が更に長く複雑になっていきます。入力層に近いパラメータになると、その前の層までの計算結果を流用できるようになり、計算量も削減することができます。このように、計算過程で共通する項目を逆方向から伝播していくと計算量の削減ができ、この方法が**誤差逆伝播法**と知られています。予測値の計算では左側から右側へ値を伝播していきましたが、この方向を順方向とし、パラメータ更新のために必要な勾配の計算が逆方向に値を伝播させるため、**逆伝播**と呼ばれています。  

第 1 、2 章で学んだ内容は次章からの実装やその他の種類のニューラルネットワークについて学ぶ際の基礎となります。前章の内容も振り返り、次の章に進むようにしましょう。  

## 練習問題 本章のまとめ

本章で学んだ内容を復習しましょう。問題の答えをそれぞれのセルに記載してください。
（プログラミングを行うのではなく、問題をノート上などで解き、回答を空白のセルに記入してください。）  

下記のニューラルネットワークの計算を行い、更新後のパラメータ $w_7$ の算出を行って下さい。  
計算内容については下記の補足を確認して下さい。

![練習問題](http://drive.google.com/uc?export=view&id=1GlMgINpkT7RUF2vFHB3ehzgkM2TLSc2b)


*補足*  

- 非線形変換には ReLU 関数を用います。
- 目的関数には平均二乗誤差を用います。
- 学習係数の値は 0.01 とします。

In [ ]:
# u11


In [ ]:
# u12


In [ ]:
# z11


In [ ]:
# z12


In [ ]:
# y


In [ ]:
# 平均二乗誤差


In [ ]:
# 勾配


In [ ]:
# 更新後のw7


---
© 株式会社キカガク及び国立大学法人 豊橋技術科学大学